<a href="https://colab.research.google.com/github/Jrajeev88/WithYourMind/blob/main/MCA_project_Final_04_01_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "transformers>=4.41.0,<6.0.0" accelerate sentencepiece datasets bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.3 MB/s eta 0:00:00


In [2]:
!pip install bitsandbytes accelerate -q

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch, os

os.makedirs("offload", exist_ok=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

BASE_MODEL = "Tanneru/Mistral-7B-Instruct-v0.3-Mental-Health-chatbot"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="offload",
    low_cpu_mem_usage=True
)

# 🔧 REQUIRED FIX FOR GRADIENTS
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()
model.train()

print("Model loaded + prepared for 4-bit LoRA fine-tuning correctly.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Model loaded + prepared for 4-bit LoRA fine-tuning correctly.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

ALIGN_MODEL = "VaisakhKrishna/Llama-2-Emotional-ChatBot"

teacher_tokenizer = AutoTokenizer.from_pretrained(ALIGN_MODEL)

# Configure 8-bit quantization with CPU offload
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

teacher_model = AutoModelForCausalLM.from_pretrained(
    ALIGN_MODEL,
    quantization_config=bnb_config, # Use the quantization configuration
    device_map="auto"
)

print("LLaMA-2 Emotional alignment model loaded.")


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

LLaMA-2 Emotional alignment model loaded.


In [4]:
sample_prompts = [
    "I feel like nobody understands me.",
    "I'm losing motivation to continue my work.",
    "My emotions feel heavy and confusing.",
    "I don't know how to handle stress anymore.",
    "I just feel exhausted and empty."
]

alignment_pairs = []

for prompt in sample_prompts:
    inputs = teacher_tokenizer(prompt, return_tensors="pt").to(device)
    output = teacher_model.generate(
        **inputs,
        max_length=200,
        temperature=0.7,
        top_p=0.9
    )
    teacher_reply = teacher_tokenizer.decode(output[0], skip_special_tokens=True)
    alignment_pairs.append({"input_text": prompt, "target_text": teacher_reply})

alignment_pairs[:3]


[{'input_text': 'I feel like nobody understands me.',
  'target_text': "I feel like nobody understands me.\nI feel like I'm being misunderstood all the time.\nI feel like I'm not being heard.\nI feel like I'm being ignored.\nI feel like I'm being dismissed.\nI feel like I'm being ridiculed.\nI feel like I'm being belittled.\nI feel like I'm being bullied.\nI feel like I'm being attacked.\nI feel like I'm being criticized.\nI feel like I'm being judged.\nI feel like I'm being scrutinized.\nI feel like I'm being looked down upon.\nI feel like I'm being discriminated against.\nI feel like I'm being ostracized.\nI feel like I'm being rejected.\nI feel like I'm being isolated.\nI feel like I'"},
 {'input_text': "I'm losing motivation to continue my work.",
  'target_text': "I'm losing motivation to continue my work. It's not that I don't enjoy it, but I feel like I'm not making progress. I'm not seeing the results I want and it's really frustrating.\n\nI've been feeling this way for a while

In [5]:
from peft import LoraConfig, get_peft_model

lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()


trainable params: 38,535,168 || all params: 7,286,558,720 || trainable%: 0.5289


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [6]:
from peft import LoraConfig, get_peft_model

lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 38,535,168 || all params: 7,286,558,720 || trainable%: 0.5289


In [7]:
from datasets import load_dataset
counsel_ds = load_dataset("Amod/mental_health_counseling_conversations")

train_counsel = counsel_ds["train"]
print("Counseling samples:", len(train_counsel))
print(train_counsel[0])


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Counseling samples: 3512
{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about thems

In [8]:
def format_counsel(batch):
    return {
        "input_text":  f"User: {batch['Context']}",
        "target_text": f"WithYourMind: {batch['Response']}"
    }

counsel_pairs = train_counsel.map(format_counsel)
counsel_pairs[0]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [9]:
def preprocess(batch):
    x = tokenizer(batch["input_text"],  truncation=True, padding="max_length", max_length=256)
    y = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=256)
    x["labels"] = y["input_ids"]
    return x

train_ready = counsel_pairs.map(preprocess, batched=True)


Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="/content/WithYourMind_CounselingTune",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=1,  # We keep it 1 first for safety
    fp16=True,
    logging_steps=50,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ready,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-1929692163.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,6.486200
100,5.357800
150,5.198800
200,5.239600
250,5.290100
300,5.172100
350,5.203300
400,5.146800


TrainOutput(global_step=439, training_loss=5.36876975648224, metrics={'train_runtime': 4449.8488, 'train_samples_per_second': 0.789, 'train_steps_per_second': 0.099, 'total_flos': 3.858281712215654e+16, 'train_loss': 5.36876975648224, 'epoch': 1.0})

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

godel_name = "microsoft/GODEL-v1_1-base-seq2seq"

godel_tokenizer = AutoTokenizer.from_pretrained(godel_name)
godel_model = AutoModelForSeq2SeqLM.from_pretrained(
    godel_name,
    device_map="auto",
    torch_dtype=torch.float16
)

print("GODEL-base model loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

GODEL-base model loaded successfully.


In [12]:
def godel_generate(context):
    prompt = f"Instruction: respond empathetically.\nContext: {context}\nResponse:"
    inputs = godel_tokenizer(prompt, return_tensors="pt").to("cuda")
    output = godel_model.generate(**inputs, max_length=200)
    return godel_tokenizer.decode(output[0], skip_special_tokens=True)

In [13]:
godel_generate("I feel stuck in life and can't move forward.")

"I'm sorry to hear that."

In [14]:
generated_pairs = []

sample_inputs = [
    "I feel stuck in life.",
    "I am tired of everything.",
    "I feel lonely even around people.",
    "I want to start over but I am scared.",
    "I do not know how to handle stress anymore."
]

for text in sample_inputs:
    reply = godel_generate(text)
    generated_pairs.append({
        "input_text": f"User: {text}",
        "target_text": f"WithYourMind: {reply}"
    })

generated_pairs[:2]


[{'input_text': 'User: I feel stuck in life.',
  'target_text': "WithYourMind: I'm sorry to hear that."},
 {'input_text': 'User: I am tired of everything.',
  'target_text': "WithYourMind: I'm tired of everything."}]

In [15]:
def preprocess_godel(batch):
    x = tokenizer(batch["input_text"],  truncation=True, padding="max_length", max_length=192)
    y = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=192)
    x["labels"] = y["input_ids"]
    return x

train_ready_godel = list(map(preprocess_godel, generated_pairs))

trainer.train_dataset = train_ready_godel
trainer.train()


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.3958256244659424, metrics={'train_runtime': 23.8987, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.042, 'total_flos': 41197484113920.0, 'train_loss': 1.3958256244659424, 'epoch': 1.0})

In [16]:
model.eval()
torch.cuda.empty_cache()
print("Model set to evaluation mode.")

Model set to evaluation mode.


In [17]:
def withyourmind_chat(user_input, history=""):
    crisis_words = ["suicide", "self harm", "kill myself", "can't live", "end my life"]

    if any(word in user_input.lower() for word in crisis_words):
        return ("I’m really glad you told me. I can’t help with emergencies, "
                "but you deserve immediate support.\n"
                "Please reach someone right now:\n"
                "• India: 9152987821 (AASRA Helpline)\n"
                "• Global: https://www.iasp.info/resources/Crisis_Centres/\n"
                "• Local emergency services if you feel unsafe.")

    prompt = f"""
    You are WithYourMind, a non-clinical emotional support companion.
    You do not diagnose, prescribe, or claim to be a professional.
    Focus only on reflective listening, empathy, emotional support, and grounding questions.

    Conversation history:
    {history}

    User: {user_input}
    WithYourMind:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_new_tokens=180,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.split("WithYourMind:")[-1].strip()

In [18]:
from peft import PeftModel # We only need PeftModel if we want to check its type or specifically use it

model = model.merge_and_unload()   # Call the method directly on the PeftModel object
model.to("cuda")
model.eval()

print("LoRA weights successfully merged. Model is now stable for inference.")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LoRA weights successfully merged. Model is now stable for inference.


In [19]:
from transformers import GenerationConfig

model.generation_config = GenerationConfig(
    max_new_tokens=180,
    temperature=0.7,
    top_p=0.85,
    do_sample=True,
    repetition_penalty=1.25,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

In [20]:
print(
    withyourmind_chat("I feel stuck and confused about life.")
)

That must be challenging for you. Have you considered making a list of your interests, skills, and values to help identify potential careers that align with who you are as an individual? Then, research those fields further to see if they offer paths in which you can envision yourself thriving. Remember, it's okay to try different things before finding the right fit. Keep exploring, learning, and growing! And always remember, there's no rush to figure everything out at once. It may take time to discover your true passion(s). Is that something helpful


In [21]:
!pip install gradio -q


In [50]:
import gradio as gr

def gradio_chat(user_input, history):
    if history is None:
        history = []

    # First, check for English crisis input
    english_crisis_response = check_crisis_status(user_input)
    if english_crisis_response:
        history.append((user_input, english_crisis_response))
        return history

    # Then, check for Malayalam input and handle its crisis/translation
    if is_malayalam(user_input):
        reply = withyourmind_ml(user_input, history)
        history.append((user_input, reply))
        return history

    # If neither English crisis nor Malayalam, generate a standard English response
    reply = withyourmind_chat(user_input, history)
    history.append((user_input, reply))
    return history

with gr.Blocks(theme="soft") as iface:
    gr.Markdown("# 🤝 WithYourMind — Emotional Support Chatbot")
    gr.Markdown("Non-clinical emotional support • Empathetic responses • Crisis-safe boundaries")

    chatbot = gr.Chatbot(height=450)
    msg = gr.Textbox(label="How are you feeling today?", placeholder="Type here...")

    msg.submit(gradio_chat, [msg, chatbot], chatbot)
    msg.submit(lambda x: "", msg)

iface.launch()

/tmp/ipython-input-797995431.py:24: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft") as iface:
/tmp/ipython-input-797995431.py:28: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=450)
/tmp/ipython-input-797995431.py:28: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=450)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b55973eb12e5cdba6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
iface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2703f763081ab733dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
!pip uninstall -y googletrans httpx
!pip install deep_translator -q

Found existing installation: googletrans 4.0.0rc1
Uninstalling googletrans-4.0.0rc1:
  Successfully uninstalled googletrans-4.0.0rc1
Found existing installation: httpx 0.13.3
Uninstalling httpx-0.13.3:
  Successfully uninstalled httpx-0.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [25]:
malayalam_chars = set("അആഇഈഉഊഋഎഏഐഒഓഔകഖഗഘങചഛജഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ")
def is_malayalam(text):
    return any(ch in malayalam_chars for ch in text)


In [51]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='auto', target='en') # Initialize once

def withyourmind_ml(user_input, history=""):
    # Crisis detection before translation
    crisis_terms = ["ആത്മഹത്യ", "ആത്മഹത്യാപരമായ", "ജീവിതം അവസാനിപ്പിക്കുക", "വേദനിക്കണം", "പോകണം"]
    if any(term in user_input for term in crisis_terms):
        return (
            "നിങ്ങൾക്ക് അത്യധികം വിഷമം ഉണ്ടെന്നു ഞാൻ മനസിലാക്കുന്നു. പക്ഷെ, അടിയന്തിര സാഹചര്യങ്ങളിൽ എനിക്ക് നിങ്ങളെ സഹായിക്കാൻ കഴിയില്ല. ഏറ്റവും അടുത്ത ഒരു ബന്ധുവിനോടോ സുഹൃത്തിനോടോ പങ്കു വയ്ക്കുകയോ അല്ലെങ്കിൽ താഴെ പറയുന്ന നമ്പറുകളിൽ വിളിക്കുകയോ ചെയ്യേണ്ടതാണ്. ഓർക്കുക, ഇത്തരം സാഹചര്യങ്ങൾ മാനുഷികമാണ്.  അതിനെ മറികടക്കാൻ വേണ്ട ശ്രമങ്ങൾ ചെയ്യുക.\n"
        "📍 **India-specific crisis support:**\n"
        "• AASRA (24/7 Suicide Prevention): 022 2754 6669\n"
        "• Govt. KIRAN Mental Health Helpline: 1800-599-0019\n"
        "• iCALL WhatsApp Support: +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
        "If you're in immediate danger, please contact a human right now.\n"
        "You are not alone.\n"
        )

    # Detect Malayalam
    if is_malayalam(user_input):
        en = translator.translate(user_input)
        response_en = withyourmind_chat(en, history)
        response_ml = GoogleTranslator(source='en', target='ml').translate(response_en)
        return response_ml

    # Fallback if not Malayalam
    return withyourmind_chat(user_input, history)

In [52]:
def gradio_chat(user_input, history):
    if history is None:
        history = []

    if is_malayalam(user_input):
        reply = withyourmind_ml(user_input)
        history.append((user_input, reply))
        return history

    reply = withyourmind_chat(user_input)
    history.append((user_input, reply))
    return history


In [33]:
def naturalize_malayalam(text):
    replacements = {
        "നിങ്ങൾ ഒരാളല്ല": "നിങ്ങൾ ഒറ്റക്കല്ല",
        "ഒരു തീരുമാനം എടുക്കണം": "ഒരു തീരുമാനത്തിലേക്ക് എത്തണം",
        "ഞാൻ ഇവിടെ നിങ്ങളോടൊപ്പം": "ഞാൻ നിങ്ങളോടൊപ്പം ഉണ്ടെന്ന് കരുതുക",
        "നിങ്ങൾക്ക് അത് ബുദ്ധിമുട്ടാകാം": "അത് വെറും എളുപ്പമല്ലെന്ന് തോന്നുന്നു",
        "കഠിനമാണ്": "ബുദ്ധിമുട്ടായി തോന്നുന്നു",
        "ആശയക്കുഴപ്പം": "കുഴപ്പം",
        "സഹായം തേടുക": "ആളുമായി സംസാരിക്കുക"
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

In [53]:
malayalam_chars = set("അആഇഈഉഊഋഎഏഐഒഓഔകഖഗഘങചഛജഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ")
def is_malayalam(text):
    return any(ch in malayalam_chars for ch in text)

crisis_terms_ml = ["ആത്മഹത്യ","ആത്മഹത്യാപരമായ","ജീവിതം അവസാനിപ്പിക്കുക","പോകണം","എനിക്കു ജീവിക്കാനില്ല"] # Define globally

def withyourmind_ml(user_input, history=""):
    # Crisis detection for Malayalam input
    if any(term in user_input for term in crisis_terms_ml):
        return emergency_response_ml() # Use the more detailed emergency response function

    # Detect Malayalam
    if is_malayalam(user_input):
        en = translator.translate(user_input)
        reply_en = withyourmind_chat(en, history)
        reply_ml = GoogleTranslator(source='en', target='ml').translate(reply_en)
        return naturalize_malayalam(reply_ml)

    # If English input
    return withyourmind_chat(user_input, history)

In [54]:
def emergency_response_ml():
    return (
        "നിങ്ങൾ പങ്കുവെച്ചത് വളരെ പ്രധാനപ്പെട്ടതാണ്. ഞാൻ ഇവിടെ കേൾക്കാൻ ആണ്, "
        "എന്നാൽ അടിയന്തിര സാഹചര്യങ്ങളിൽ നേരിട്ട് സഹായം നൽകാൻ എനിക്ക് കഴിയില്ല.\n\n"

        "📍 **ഇന്ത്യയിലെ അടിയന്തിര സഹായ നമ്പറുകൾ:**\n"
        "• AASRA (24/7): 022 2754 6669\n"
        "• സർക്കാർ KIRAN ഹെൽപ്ലൈൻ: 1800-599-0019\n"
        "• iCALL (WhatsApp/ചാറ്റ്): +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
        "📌 ദയവായി ഒരു മനുഷ്യനുമായി ഇപ്പോൾ തന്നെ ബന്ധപ്പെടുക.\n"
        "📌 നിങ്ങൾ ഒറ്റക്കല്ല.\n"
    )

In [55]:
def emergency_response_en():
    return (
        "Thank you for saying that. I’m here to listen, but I can’t provide emergency help.\n\n"
        "📍 **India-specific crisis support:**\n"
        "• AASRA (24/7 Suicide Prevention): 022 2754 6669\n"
        "• Govt. KIRAN Mental Health Helpline: 1800-599-0019\n"
        "• iCALL WhatsApp Support: +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
        "If you're in immediate danger, please contact a human right now.\n"
        "You are not alone.\n"
    )

In [56]:
def check_crisis_status(user_input):
    crisis_terms = [
        "suicide", "kill myself", "end my life", "self harm",
        "can't go on", "want to die", "hopeless", "worthless",
        "no reason to live", "take my life", "harm myself", "give up",
        "feel like dying", "don't want to live", "better off dead",
        "suicidal thoughts", "ending it all"
    ]

    if any(w in user_input.lower() for w in crisis_terms):
        return emergency_response_en()
    return None # Return None if no crisis is detected

In [57]:
def withyourmind_chat(user_input, history=""):
    crisis_words = ["suicide","self harm","kill myself","end my life","can't live"]

    if any(w in user_input.lower() for w in crisis_words):
        return ("I’m sorry you feel this way. I can’t help in emergencies, but you are not alone.\n"
                "Please reach immediate human support:\n"
                "📍 **India-specific crisis support:**\n"
        "• AASRA (24/7 Suicide Prevention): 022 2754 6669\n"
        "• Govt. KIRAN Mental Health Helpline: 1800-599-0019\n"
        "• iCALL WhatsApp Support: +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
       )

    prompt = f"""
You are WithYourMind, a non-clinical emotional support companion.
Respond with empathy, reflection, and grounding. Do not diagnose or claim professional authority.

Conversation so far:
{history}

User: {user_input}
WithYourMind:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=model.generation_config.temperature,
        top_p=model.generation_config.top_p,
        repetition_penalty=model.generation_config.repetition_penalty
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.split("WithYourMind:")[-1].strip()

In [58]:
import os

save_directory = "./fine_tuned_model"
os.makedirs(save_directory, exist_ok=True)

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to ./fine_tuned_model


You can load your saved model and tokenizer in a new session using the following code:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

load_directory = "./fine_tuned_model"

tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForCausalLM.from_pretrained(load_directory)
model.to("cuda") # Move to GPU if available
model.eval()

print("Model and tokenizer loaded successfully.")
```

You'll need to create a new directory structure in your repository: `.github/workflows/`. Inside this directory, you'll place a YAML file (e.g., `deploy.yml`) that defines your workflow.

Here's an example `deploy.yml` content for deploying to Hugging Face Spaces. This workflow will:
1.  **Trigger** on pushes to the `main` branch.
2.  **Checkout** your code.
3.  **Set up Python**.
4.  **Install dependencies** from your `requirements.txt`.
5.  **Log in to Hugging Face** using a `HF_TOKEN` secret (you'll need to add this to your GitHub repository secrets).
6.  **Deploy** the Space using `huggingface-cli`.

In [66]:
%%writefile .github/workflows/deploy.yml
name: Deploy to Hugging Face Spaces

on:
  push:
    branches:
      - main

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
      - name: Checkout repository
        uses: actions/checkout@v3

      - name: Setup Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.9' # Or your preferred Python version

      - name: Install dependencies
        run: |
          pip install --upgrade pip
          pip install -r requirements.txt
          pip install huggingface_hub

      - name: Login to Hugging Face
        env:
          HF_TOKEN: ${{ secrets.HF_TOKEN }}
        run: huggingface-cli login --token $HF_TOKEN

      - name: Deploy Space
        run:
          # This command assumes your `app.py` and `fine_tuned_model` are in the root directory
          # If your space is already created, this command will update it.
          # Replace 'your-username/your-space-name' with your actual Hugging Face Space ID.
          # Example: huggingface-cli upload jrajeev/WithYourMind .
          huggingface-cli upload --repo-id Jrajeev88/WithYourMind .

Writing .github/workflows/deploy.yml


In [65]:
import os
os.makedirs('.github/workflows', exist_ok=True)

First, we'll create the `app.py` file, which will contain all the necessary Python code for your chatbot, including model loading, translation functions, crisis detection, and the Gradio interface setup. Make sure the `fine_tuned_model` directory is present in the same location as `app.py` when you deploy or run it elsewhere.

In [59]:
%%writefile app.py
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from deep_translator import GoogleTranslator
import torch
import os

# Create offload directory if it doesn't exist, needed by device_map="auto"
os.makedirs("offload", exist_ok=True)

# --- Model Loading ---
# Path to your fine-tuned model. Make sure this directory contains
# your saved model (e.g., adapter_model.bin) and tokenizer files.
# If deploying to Hugging Face Spaces, you would typically upload this directory
# as part of your repository, or specify a Hugging Face model ID.
FINE_TUNED_MODEL_PATH = "./fine_tuned_model"

# Configure BitsAndBytes for 4-bit quantization if your saved model is quantized.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
    llm_int8_enable_fp32_cpu_offload=True # Needed if parts are offloaded to CPU
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    FINE_TUNED_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto", # Automatically distributes model layers across available devices
    offload_folder="offload", # Directory for CPU offloading
    low_cpu_mem_usage=True # Reduces CPU RAM usage during loading
)
model.eval() # Set model to evaluation mode
if torch.cuda.is_available():
    model.to("cuda") # Ensure model is on GPU if available

# Set generation configuration for consistent outputs
model.generation_config = GenerationConfig(
    max_new_tokens=180,
    temperature=0.7,
    top_p=0.85,
    do_sample=True,
    repetition_penalty=1.25,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# --- Translation and Crisis Functions ---
translator = GoogleTranslator(source='auto', target='en')

malayalam_chars = set("അആഇഈഉഊഋഎഏഐഒഓഔകഖഗഘങചഛജഞടഠഡഢണതഥദധനപഫബഭമയരലവശഷസഹളഴറ")
def is_malayalam(text):
    return any(ch in malayalam_chars for ch in text)

def emergency_response_ml():
    return (
        "നിങ്ങൾ പങ്കുവെച്ചത് വളരെ പ്രധാനപ്പെട്ടതാണ്. ഞാൻ ഇവിടെ കേൾക്കാൻ ആണ്, "
        "എന്നാൽ അടിയന്തിര സാഹചര്യങ്ങളിൽ നേരിട്ട് സഹായം നൽകാൻ എനിക്ക് കഴിയില്ല.\n\n"
        "📍 **ഇന്ത്യയിലെ അടിയന്തിര സഹായ നമ്പറുകൾ:**\n"
        "• AASRA (24/7): 022 2754 6669\n"
        "• സർക്കാർ KIRAN ഹെൽപ്ലൈൻ: 1800-599-0019\n"
        "• iCALL (WhatsApp/ചാറ്റ്): +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
        "📌 ദയവായി ഒരു മനുഷ്യനുമായി ഇപ്പോൾ തന്നെ ബന്ധപ്പെടുക.\n"
        "📌 നിങ്ങൾ ഒറ്റക്കല്ല.\n"
    )

def emergency_response_en():
    return (
        "Thank you for saying that. I’m here to listen, but I can’t provide emergency help.\n\n"
        "📍 **India-specific crisis support:**\n"
        "• AASRA (24/7 Suicide Prevention): 022 2754 6669\n"
        "• Govt. KIRAN Mental Health Helpline: 1800-599-0019\n"
        "• iCALL WhatsApp Support: +91 9152987821\n\n"
        ". Tele Manas: 14416\n"
        "If you're in immediate danger, please contact a human right now.\n"
        "You are not alone.\n"
    )

def check_crisis_status(user_input):
    crisis_terms = [
        "suicide", "kill myself", "end my life", "self harm",
        "can't go on", "want to die", "hopeless", "worthless",
        "no reason to live", "take my life", "harm myself", "give up",
        "feel like dying", "don't want to live", "better off dead",
        "suicidal thoughts", "ending it all"
    ]

    if any(w in user_input.lower() for w in crisis_terms):
        return emergency_response_en()
    return None

def naturalize_malayalam(text):
    replacements = {
        "നിങ്ങൾ ഒരാളല്ല": "നിങ്ങൾ ഒറ്റക്കല്ല",
        "ഒരു തീരുമാനം എടുക്കണം": "ഒരു തീരുമാനത്തിലേക്ക് എത്തണം",
        "ഞാൻ ഇവിടെ നിങ്ങളോടൊപ്പം": "ഞാൻ നിങ്ങളോടൊപ്പം ഉണ്ടെന്ന് കരുതുക",
        "നിങ്ങൾക്ക് അത് ബുദ്ധിമുട്ടാകാം": "അത് വെറും എളുപ്പമല്ലെന്ന് തോന്നുന്നു",
        "കഠിനമാണ്": "ബുദ്ധിമുട്ടായി തോന്നുന്നു",
        "ആശയക്കുഴപ്പം": "കുഴപ്പം",
        "സഹായം തേടുക": "ആളുമായി സംസാരിക്കുക"
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

def withyourmind_chat(user_input, history=""):
    prompt = f"""
You are WithYourMind, a non-clinical emotional support companion.\nRespond with empathy, reflection, and grounding. Do not diagnose or claim professional authority.\n\nConversation so far:\n{history}\n\nUser: {user_input}\nWithYourMind:\n""".strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=model.generation_config.temperature,
        top_p=model.generation_config.top_p,
        repetition_penalty=model.generation_config.repetition_penalty
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.split("WithYourMind:")[-1].strip()

def withyourmind_ml(user_input, history=""):
    crisis_terms_ml = ["ആത്മഹത്യ","ആത്മഹത്യാപരമായ","ജീവിതം അവസാനിപ്പിക്കുക","പോകണം","എനിക്കു ജീവിക്കാനില്ല"]
    if any(term in user_input for term in crisis_terms_ml):
        return emergency_response_ml()

    if is_malayalam(user_input):
        en = translator.translate(user_input)
        reply_en = withyourmind_chat(en, history)
        reply_ml = GoogleTranslator(source='en', target='ml').translate(reply_en)
        return naturalize_malayalam(reply_ml)
    return withyourmind_chat(user_input, history)

# --- Gradio Interface Function ---
def gradio_chat(user_input, history):
    if history is None:
        history = []

    english_crisis_response = check_crisis_status(user_input)
    if english_crisis_response:
        history.append((user_input, english_crisis_response))
        return history

    if is_malayalam(user_input):
        reply = withyourmind_ml(user_input, history)
        history.append((user_input, reply))
        return history

    reply = withyourmind_chat(user_input, history)
    history.append((user_input, reply))
    return history

# --- Gradio Blocks Setup ---
if __name__ == "__main__":
    with gr.Blocks(theme="soft") as iface:
        gr.Markdown("# 🤝 WithYourMind — Emotional Support Chatbot")
        gr.Markdown("Non-clinical emotional support • Empathetic responses • Crisis-safe boundaries")

        chatbot = gr.Chatbot(height=450)
        msg = gr.Textbox(label="How are you feeling today?", placeholder="Type here...")

        msg.submit(gradio_chat, [msg, chatbot], chatbot)
        msg.submit(lambda x: "", msg)

    iface.launch(share=True)

Overwriting app.py


Next, let's create a `requirements.txt` file, which lists all the Python libraries your project depends on. This is crucial for anyone else (or yourself) to set up the environment and run your code.

In [60]:
%%writefile requirements.txt
transformers
accelerate
bitsandbytes
peft
torch
gradio
deep-translator
sentencepiece

Overwriting requirements.txt


Now, we'll create a `README.md` file. This file provides essential information about your project, how to use it, its features, and any disclaimers.

In [61]:
%%writefile README.md
# WithYourMind - Emotional Support Chatbot

This is an emotional support chatbot built using a fine-tuned Mistral-7B-Instruct model, capable of providing empathetic responses in both English and Malayalam. It also includes robust crisis detection and provides region-specific helpline information for India.

## Features:
- Empathetic conversational AI
- Supports English and Malayalam languages
- Crisis detection with immediate helpline referrals (India-specific)
- Built with Hugging Face Transformers, PEFT, and Gradio

## How to use:
1. Clone this repository.
2. Ensure you have the `fine_tuned_model` directory (containing the saved model and tokenizer) in the same location as `app.py`.
3. Install the required dependencies: `pip install -r requirements.txt`
4. Run the Gradio application: `python app.py`

## Crisis Information:
If you or someone you know is in crisis, please seek immediate help:
-   **India-specific crisis support:**
    -   AASRA (24/7 Suicide Prevention): 022 2754 6669
    -   Govt. KIRAN Mental Health Helpline: 1800-599-0019
    -   iCALL WhatsApp Support: +91 9152987821
    -   Tele Manas: 14416
-   **Global:** Visit https://www.iasp.info/resources/Crisis_Centres/ for a list of international helplines.

## Disclaimer:
This chatbot is a non-clinical emotional support companion. It does not diagnose, prescribe, or provide professional medical or psychological advice. It is not a substitute for professional help. In emergencies, please contact qualified professionals or emergency services.

Overwriting README.md


Finally, we'll create a `.gitignore` file to tell Git which files and directories to ignore, such as cache folders, temporary files, and especially large model checkpoints if you plan to host the model elsewhere (e.g., Hugging Face Hub) instead of directly in your GitHub repo.

In [62]:
%%writefile .gitignore
__pycache__/
*.pyc
*.bin
*.pt
*.pth
*.safetensors
token*
apikey*
offload/

Overwriting .gitignore
